<a href="https://colab.research.google.com/github/dimitri-mar/CorrelAidX2022Challenge/blob/main/Explore_OffshoreLeaks_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python for Offshore Leaks: Explore the dataset and connect to Graph Database

We can download the csv files as zip. 

In [1]:
!wget https://offshoreleaks-data.icij.org/offshoreleaks/csv/full-oldb.20221031.zip

--2022-11-17 13:42:56--  https://offshoreleaks-data.icij.org/offshoreleaks/csv/full-oldb.20221031.zip
Resolving offshoreleaks-data.icij.org (offshoreleaks-data.icij.org)... 13.225.103.44, 13.225.103.23, 13.225.103.8, ...
Connecting to offshoreleaks-data.icij.org (offshoreleaks-data.icij.org)|13.225.103.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71275117 (68M) [application/zip]
Saving to: ‘full-oldb.20221031.zip’

full-oldb.20221031. 100%[===================>]  67.97M  16.4MB/s    in 4.9s    

2022-11-17 13:43:02 (13.9 MB/s) - ‘full-oldb.20221031.zip’ saved [71275117/71275117]



And unzip them:

In [2]:
!unzip -o full-oldb.20221031.zip  


Archive:  full-oldb.20221031.zip
  inflating: nodes-entities.csv      
  inflating: nodes-officers.csv      
  inflating: nodes-addresses.csv     
  inflating: nodes-intermediaries.csv  
  inflating: nodes-others.csv        
  inflating: relationships.csv       


In [3]:
import pandas as pd
import networkx as nx
from glob import glob
from IPython.display import display

# Relationships

In [10]:
# let's load the relationships as a dataframe
relationships = pd.read_csv("relationships.csv")
relationships.sample(25, random_state=1234)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,node_id_start,node_id_end,rel_type,link,status,start_date,end_date,sourceID
2524008,56066672,55007576,officer_of,director of,NaN,NaN,NaN,Paradise Papers - Malta corporate registry
2349115,230000024,200121452,intermediary_of,registered agent of,NaN,NaN,NaN,NaN
2177289,130000006,101513567,intermediary_of,registered agent of,NaN,NaN,NaN,NaN
1119215,21132,197807,officer_of,director of,NaN,2006-03-07,NaN,Offshore Leaks
537233,20093194,10212426,same_company_as,same company as,NaN,NaN,NaN,Bahamas Leaks
1329053,23000249,20019051,intermediary_of,intermediary of,NaN,NaN,NaN,Bahamas Leaks
1309131,23000372,20006425,intermediary_of,intermediary of,NaN,NaN,NaN,Bahamas Leaks
3285808,240510035,240513025,same_name_as,same name as,NaN,NaN,NaN,NaN
3125311,56035142,58039969,registered_address,registered address,NaN,NaN,NaN,Paradise Papers - Malta corporate registry
255708,12130870,10107914,officer_of,shareholder of,NaN,16-FEB-2004,NaN,Panama Papers


In [11]:
rel.shape

(3336913, 8)

In [6]:
full_graph = nx.from_pandas_edgelist(rel, source="node_id_start", target="node_id_end", edge_attr=True, create_using=nx.DiGraph())

In [7]:
type(full_graph)

networkx.classes.digraph.DiGraph

# Nodes

In [8]:
node_files = glob("nodes*.csv")
# extract node type
node_types = {nf[6:-4]:nf for nf in node_files}
node_types

{'intermediaries': 'nodes-intermediaries.csv',
 'addresses': 'nodes-addresses.csv',
 'entities': 'nodes-entities.csv',
 'others': 'nodes-others.csv',
 'officers': 'nodes-officers.csv'}

In [24]:
# comment it in case there is not enough memory
node_df_dict = {node_type: pd.read_csv(node_types[node_type]) for node_type in node_types }

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (2,7) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (2,3,6,7,10,11,12,13,14,15,16,17,20) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [9]:
for node_type in node_types:
  print(node_type)
  display(pd.read_csv(node_types[node_type]).sample(5, random_state=1234))

intermediaries


,node_id,name,status,internal_id,address,countries,country_codes,sourceID,valid_until,note
14072,11014072,"NORTHRUP, RONALD",SUSPENDED,9562,MR. RONALD NORTHRUP HOSPITAL THERAPY SERVICES ...,United States,USA,Panama Papers,The Panama Papers data is current through 2015,NaN
3439,11003445,JACQUELINE LOH,SUSPENDED,20823,JACQUELINE LOH SUITE 4203 TWO EXCHANGE SQUARE ...,Hong Kong,HKG,Panama Papers,The Panama Papers data is current through 2015,NaN
4812,11004808,EDI MICHELLINI,ACTIVE,26620,NaN,Guatemala,GTM,Panama Papers,The Panama Papers data is current through 2015,NaN
6939,11006940,MATT SMITH,NaN,33501,NaN,NaN,NaN,Panama Papers,The Panama Papers data is current through 2015,NaN
11749,11011750,THE SINCERE COMPANY LIMITED,ACTIVE,7583,"THE SINCERE COMPANY LIMITED 24/F., LEIGHTON CE...",Hong Kong,HKG,Panama Papers,The Panama Papers data is current through 2015,NaN


addresses


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,node_id,address,name,countries,country_codes,sourceID,valid_until,note
56876,14056871,NO. 46; XI ZU; WAZHANG VILLAGE; XINYI COUNTY; ...,NaN,China,CHN,Panama Papers,The Panama Papers data is current through 2015,NaN
268212,58143932,"'ELEON' TAL-IBRAGG ROAD, TAL-IBRAG SWIEQI","'ELEON' TAL-IBRAGG ROAD, TAL-IBRAG SWIEQI",NaN,NaN,Paradise Papers - Malta corporate registry,Malta corporate registry data is current throu...,NaN
381976,58134140,"FL 2, GEMELLE COURT, TRIQ IS-SIDRA, SWIEQI","FL 2, GEMELLE COURT, TRIQ IS-SIDRA, SWIEQI",NaN,NaN,Paradise Papers - Malta corporate registry,Malta corporate registry data is current throu...,NaN
323389,58063335,"EASYSELL SHOWROOM, MDINA ROAD, QORMI, MALTA","EASYSELL SHOWROOM, MDINA ROAD, QORMI, MALTA",NaN,NaN,Paradise Papers - Malta corporate registry,Malta corporate registry data is current throu...,NaN
379666,58131249,"92, ALBA, TRIQ IL-MEDITERRAN, PEMBROKE PBK1251","92, ALBA, TRIQ IL-MEDITERRAN, PEMBROKE PBK1251",NaN,NaN,Paradise Papers - Malta corporate registry,Malta corporate registry data is current throu...,NaN


...

,node_id,name,original_name,former_name,jurisdiction,jurisdiction_description,company_type,address,internal_id,incorporation_date,...,struck_off_date,dorm_date,status,service_provider,ibcRUC,country_codes,countries,sourceID,valid_until,note
747711,55032020,J X G LIMITED,NaN,NaN,MLT,Malta,NaN,NaN,NaN,30-MAY-2006,...,NaN,NaN,NaN,NaN,C38816,MLT,Malta,Paradise Papers - Malta corporate registry,Malta corporate registry data is current throu...,NaN
616243,101700801,GROWTH EQUITY (BARBADOS) LIMITED,NaN,NaN,BRB,Barbados,NaN,NaN,NaN,14-NOV-1985,...,NaN,NaN,NaN,NaN,NaN,BRB,Barbados,Paradise Papers - Barbados corporate registry,Barbados corporate registry data is current th...,NaN
367891,20056574,PACIFIC EAST INC.,PACIFIC EAST INC.,NaN,BAH,Bahamas,NaN,NaN,56574.0,11-APR-1997,...,10-APR-2003,NaN,NaN,NaN,56574B,NaN,NaN,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
589005,100302629,FIBERPOL MANUFACTURING COMPANY LIMITED,NaN,NaN,BRB,Barbados,NaN,NaN,NaN,15-JAN-1976,...,NaN,NaN,NaN,NaN,NaN,BRB,Barbados,Paradise Papers - Barbados corporate registry,Barbados corporate registry data is current th...,NaN
329576,20007198,HYDROCARBONS CONSTRUCTION INTERNATIONAL S.A.,HYDROCARBONS CONSTRUCTION INTERNATIONAL S.A.,NaN,BAH,Bahamas,NaN,NaN,7198.0,19-NOV-1991,...,31-DEC-1994,NaN,NaN,NaN,7198B,NaN,NaN,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN


others


,node_id,name,type,incorporation_date,struck_off_date,closed_date,jurisdiction,jurisdiction_description,countries,country_codes,sourceID,valid_until,note
406,80021806,Smit Salvage Limited,NaN,NaN,NaN,NaN,NaN,NaN,Isle of Man,IMN,Paradise Papers - Appleby,Appleby data is current through 2014,NaN
2948,240558030,"El Cuate Holdings, Inc",NaN,NaN,NaN,NaN,PAN,Panama,NaN,NaN,Pandora Papers - Overseas Management Company (...,Provider data is current through 1970,NaN
1106,80072520,Group - David Meyerhoff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paradise Papers - Appleby,Appleby data is current through 2014,NaN
1507,80074706,Harbor Sky LTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paradise Papers - Appleby,Appleby data is current through 2014,NaN
2504,85027703,ROYAL DEN GROUP N.V.,LIMITED LIABILITY COMPANY,16-JAN-2001,NaN,NaN,AW,Aruba,NaN,NaN,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,NaN


officers


,node_id,name,countries,country_codes,sourceID,valid_until,note
236215,13000816,JAVIER ANDRES ORDEN CIRISOLA,Uruguay,URY,Panama Papers,The Panama Papers data is current through 2015,NaN
484693,110005687,HAYNES SEDRICK COLERIDGE,NaN,NaN,Paradise Papers - Barbados corporate registry,Barbados corporate registry data is current th...,NaN
745695,240052227,DIEGO JOSE FERNANDEZ KLOSE,Guatemala,GTM,"Pandora Papers - Alemán, Cordero, Galindo & Le...",Provider data is current through 2017,NaN
759933,240470990,Vanessa Marie ANTOINE PAYET,NaN,NaN,Pandora Papers - SFM Corporate Services,Provider data is current through 2016,NaN
674392,56057940,Alexander Dudarenko,Russia,RUS,Paradise Papers - Malta corporate registry,Malta corporate registry data is current throu...,NaN


In [26]:
# we can create a map to go from the node_id to its label(s)

map_list = [pd.Series(node_type, index=node_df_dict[node_type]["node_id"]) for node_type in node_df_dict]
map_node_type = pd.concat(map_list, axis=0)
map_node_type.sample(20)

node_id
81024544          addresses
81049736          addresses
20157309           entities
11013385     intermediaries
240000099         addresses
14034990          addresses
110051913          officers
240380630          officers
55050714           entities
81044634          addresses
12180929           officers
55016799           entities
12128022           officers
85008075           entities
170222             entities
291006       intermediaries
110098630          officers
10129765           entities
12215953           officers
82009730           entities
dtype: object

In [27]:
type(map_node_type)

pandas.core.series.Series

In [29]:
map_node_type.shape

(2017662,)

In [34]:
map_node_type.index.duplicated().sum()

1139

In [38]:
map_node_type[map_node_type.index.duplicated(keep=False)].sort_index()

node_id
21309     intermediaries
21309           officers
21945           officers
21945     intermediaries
24588           officers
               ...      
119071    intermediaries
119095          officers
119095    intermediaries
119104          officers
119104    intermediaries
Length: 2278, dtype: object

In [42]:
map_node_type_lst = map_node_type.to_frame().reset_index().groupby("node_id").agg(list)[0]

# Degree properties of the Offshore data
Some global characteristics are relatively easier/faster in Python. 

Fore example, we want to see what are the nodes with the largest number of connections. 

In [13]:
relationships.sample(3, random_state=234)

,node_id_start,node_id_end,rel_type,link,status,start_date,end_date,sourceID
1378241,23000280,20071900,intermediary_of,intermediary of,NaN,NaN,NaN,Bahamas Leaks
387739,12140268,10153215,officer_of,shareholder of,NaN,23-MAY-2007,18-OCT-2007,Panama Papers
2876475,56097738,55056237,officer_of,judicial representative of,NaN,NaN,NaN,Paradise Papers - Malta corporate registry


In [15]:
in_degree = relationships["node_id_end"].value_counts()
in_degree.head()

236724       37338
240000001    11962
81027146      9268
81027090      8558
81029389      6120
Name: node_id_end, dtype: int64

In [16]:
out_degree = relationships["node_id_start"].value_counts()
out_degree.head()

54662        36373
230000018    35359
23000136     14902
23000147      9719
23000330      8302
Name: node_id_start, dtype: int64

In [43]:
nodes_features = pd.DataFrame({ "in_degree": in_degree, "out_degree": out_degree}).fillna(0)
nodes_features["tot_degree"] = nodes_features["in_degree"] + nodes_features["out_degree"] 
nodes_features["node_type"] = map_node_type_lst
nodes_features.sort_values("tot_degree", ascending=False ).head(10)

,in_degree,out_degree,tot_degree,node_type
236724,37338.0,0.0,37338.0,[addresses]
54662,1.0,36373.0,36374.0,"[intermediaries, officers]"
230000018,0.0,35359.0,35359.0,[intermediaries]
23000136,0.0,14902.0,14902.0,[intermediaries]
240000001,11962.0,0.0,11962.0,[addresses]
23000147,0.0,9719.0,9719.0,[intermediaries]
81027146,9268.0,0.0,9268.0,[addresses]
81027090,8558.0,0.0,8558.0,[addresses]
23000330,0.0,8302.0,8302.0,[intermediaries]
23000156,0.0,8287.0,8287.0,[intermediaries]


# Connect to Neo4j

In [45]:
!pip install neo4j-driver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 4.1 MB/s 
  Created wheel for neo4j-driver: filename=neo4j_driver-5.2.1-py3-none-any.whl size=249609 sha256=ce542377063955151738df23330dfb42bdf8eb855afc960bd26524abb5a0c616
  Stored in directory: /root/.cache/pip/wheels/3c/64/aa/f4ccc5a3a7ca7dad493160962f43a0273465cbfc8925cb8b32
Successfully built neo4j-driver


In [51]:
# from sandbox

from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "neo4j://44.202.12.248:7687",
  auth=basic_auth("neo4j", "lead-benefits-asterisks"))

cypher_query = '''
MATCH (a:Officer {name:$name})-[r:officer_of|intermediary_of|registered_address*..10]-(b)
RETURN b.name as name, b.node_id as node_id LIMIT 20
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      name="Ross, Jr. - Wilbur Louis").data())
  #for record in results:
  #  print(record)
  df = pd.DataFrame().from_records(results)

driver.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: read_transaction has been renamed to execute_read
  app.launch_new_instance()


In [52]:
df.head()

,name,b.node_id
0,"Euro Wagon II, Ltd.",82022313
1,Gobin - Ian,80070364
2,Sciens FOF Offshore Holdings Ltd.,82022503
3,Taber - Matthew,80104550
4,Eunomia Asset Management Limited,82019517
